# YOLOv3 커스텀 데이터 학습

copy from https://articlearn.id/article/e5571bb0-how-to-train-yolov3-on-google-colab-to-de/

# 레이블링 데이터 위치

https://github.com/dhrim/deep_learning_data/raw/master/Raccoon_yolov3_darknet_format.zip

구글 드라이브 tmp/ 폴더에 Raccoon.zip 데이터 파일을 준비한다.






# 0. 임포트

In [ ]:
import cv2
import matplotlib.pyplot as plt
import glob


# 1. 구글 드라이브 마운트, mydrive로 링크

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ln -s /content/gdrive/MyDrive/ /mydrive

# 2. Darknet 다운받아서 컴파일

## 프로젝트 다운로드

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15386, done.
remote: Total 15386 (delta 0), reused 0 (delta 0), pack-reused 15386
Receiving objects: 100% (15386/15386), 14.01 MiB | 22.59 MiB/s, done.
Resolving deltas: 100% (10346/10346), done.


OpenCV, GPU, CuDNN 사용 관련 설정하고 컴파일

In [ ]:
%cd darknet

/content/darknet


## 컴파일 설정 파일 수정

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

## 컴파일

In [ ]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

# 3. 데이터 준비
- darknet/data/obj에 latops_labeled.zip 압축 풀기
- classes.names와 training.data 파일 생성
- training.txt 파일 생성


In [ ]:
# !ls -al /mydrive/tmp/laptops_labeled.zip
!ls -al /mydrive/tmp/Raccoon.zip

-rw------- 1 root root 5997816 Jan  6 01:40 /mydrive/tmp/Raccoon.zip


## data/obj/laptops_labeled 폴더에 압축 풀기

In [ ]:
!rm -rf data/obj
!mkdir data/obj
# !unzip /mydrive/tmp/laptops_labeled.zip -d data/obj
!unzip /mydrive/tmp/Raccoon.zip -d data/obj/Raccoon

Archive:  /mydrive/tmp/Raccoon.zip
   creating: data/obj/Raccoon/test/
  inflating: data/obj/Raccoon/test/_darknet.labels  
  inflating: data/obj/Raccoon/test/raccoon-57_jpg.rf.26f3fae218f26088f6a7405c58ae8020.jpg  
  inflating: data/obj/Raccoon/test/raccoon-57_jpg.rf.26f3fae218f26088f6a7405c58ae8020.txt  
  inflating: data/obj/Raccoon/test/raccoon-58_jpg.rf.ff9de9977475e0086e7ab582122c5167.jpg  
  inflating: data/obj/Raccoon/test/raccoon-58_jpg.rf.ff9de9977475e0086e7ab582122c5167.txt  
  inflating: data/obj/Raccoon/test/raccoon-59_jpg.rf.388d6dd8c9d5a6fc7ca75790e680c269.jpg  
  inflating: data/obj/Raccoon/test/raccoon-59_jpg.rf.388d6dd8c9d5a6fc7ca75790e680c269.txt  
  inflating: data/obj/Raccoon/test/raccoon-60_jpg.rf.cff8b797e6b5c9b716b6be70be2b6b33.jpg  
  inflating: data/obj/Raccoon/test/raccoon-60_jpg.rf.cff8b797e6b5c9b716b6be70be2b6b33.txt  
  inflating: data/obj/Raccoon/test/raccoon-61_jpg.rf.58c4f561336525f59d395ce21508523b.jpg  
  inflating: data/obj/Raccoon/test/raccoon-61_jp

## 학습 위한 설정 파일 생성

학습에 직접 사용할 data/obj.names, data/obj.data, data/train.txt 파일을 생성한다.

<br>

data/obj.names
```
laptop
keyboard
```

<br>

data/obj.data
```
classes = 80
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = /mydrive/tmp/custom_object_detection
```

<br>

data/train.txt
```
data/obj/laptops_labeled/8.jpg
data/obj/laptops_labeled/10.jpg
data/obj/laptops_labeled/12.jpg
data/obj/laptops_labeled/2.jpg
data/obj/laptops_labeled/6.jpg
data/obj/laptops_labeled/11.jpg
data/obj/laptops_labeled/1.jpg
data/obj/laptops_labeled/7.jpg
data/obj/laptops_labeled/5.jpg
data/obj/laptops_labeled/9.jpg
data/obj/laptops_labeled/3.jpg
data/obj/laptops_labeled/4.jpg
...
```



### data/obj.names

In [ ]:
%%writefile data/obj.names
Racoon

Writing data/obj.names


In [ ]:
!cat data/obj.names

Racoon

### data/obj.data

In [ ]:
%%writefile data/obj.data
classes = 80
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = /mydrive/tmp

Writing data/obj.data


In [ ]:
!cat data/obj.data

classes = 80
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = /mydrive/tmp

### data/train.txt


In [ ]:
!ls -al data/obj/Raccoon/train

total 5296
drwxr-xr-x 2 root root 36864 Jan  6  2022 .
drwxr-xr-x 5 root root  4096 Jan  6 01:47 ..
-rw-r--r-- 1 root root 29260 Jun  8  2021 raccoon-100_jpg.rf.e9338202a924dc489ffd94e0291a0f39.jpg
-rw-r--r-- 1 root root    78 Jun  8  2021 raccoon-100_jpg.rf.e9338202a924dc489ffd94e0291a0f39.txt
-rw-r--r-- 1 root root 34570 Jun  8  2021 raccoon-101_jpg.rf.a9e8a95f51d60d7d3b40d3243aa11afd.jpg
-rw-r--r-- 1 root root    78 Jun  8  2021 raccoon-101_jpg.rf.a9e8a95f51d60d7d3b40d3243aa11afd.txt
-rw-r--r-- 1 root root 30824 Jun  8  2021 raccoon-102_jpg.rf.d140a1fcbfc8d62c7ad37167714f79df.jpg
-rw-r--r-- 1 root root    78 Jun  8  2021 raccoon-102_jpg.rf.d140a1fcbfc8d62c7ad37167714f79df.txt
-rw-r--r-- 1 root root 45941 Jun  8  2021 raccoon-103_jpg.rf.7148e5c7d785760fe9cb6b7762180977.jpg
-rw-r--r-- 1 root root    77 Jun  8  2021 raccoon-103_jpg.rf.7148e5c7d785760fe9cb6b7762180977.txt
-rw-r--r-- 1 root root 19239 Jun  8  2021 raccoon-104_jpg.rf.e5f8de5f977a32d8a7f2223574ff2b72.jpg
-rw-r--r-- 1 root 

In [ ]:
# ls -al data/obj/laptops_labeled 밑에 jpg 파일들이 있으면
# images_list = glob.glob("data/obj/laptops_labeled/*[jpg|png|jpeg]")
images_list = glob.glob("data/obj/Raccoon/train/*[jpg|png|jpeg]")

# ls -al data/obj 밑에 jpg 파일들이 있으면
# images_list = glob.glob("data/obj/*[jpg|png|jpeg]")

print(images_list)

# Create train.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

['data/obj/Raccoon/train/raccoon-183_jpg.rf.ec95835ea04533da42d4d959d6707f20.jpg', 'data/obj/Raccoon/train/raccoon-134_jpg.rf.17aaee25817c82dcccc9982cf2346af1.jpg', 'data/obj/Raccoon/train/raccoon-131_jpg.rf.abdd5f18ff49a86f887e2de9fb682bfa.jpg', 'data/obj/Raccoon/train/raccoon-28_jpg.rf.7b2be1128d5b1d3d43776e878d595a10.jpg', 'data/obj/Raccoon/train/raccoon-171_jpg.rf.e00ec73c6a64a07c1bc0035fa0fbca18.jpg', 'data/obj/Raccoon/train/raccoon-45_jpg.rf.fa6b0fabef2ed4f9e9dd6b164792d373.jpg', 'data/obj/Raccoon/train/raccoon-26_jpg.rf.ab207a81a546038c33a728a96bf86d14.jpg', 'data/obj/Raccoon/train/raccoon-137_jpg.rf.a67b07d859ac058643915ff7161e6c90.jpg', 'data/obj/Raccoon/train/raccoon-100_jpg.rf.e9338202a924dc489ffd94e0291a0f39.jpg', 'data/obj/Raccoon/train/raccoon-119_jpg.rf.72a18cd6c7a2b03e30d1088780ddad94.jpg', 'data/obj/Raccoon/train/raccoon-182_jpg.rf.0ee4a4d4fa828093cffc43bc3ea07a8e.jpg', 'data/obj/Raccoon/train/raccoon-21_jpg.rf.1a912c8d6424050610135e493cfb5181.jpg', 'data/obj/Raccoon/t

In [ ]:
!cat data/train.txt

data/obj/Raccoon/train/raccoon-183_jpg.rf.ec95835ea04533da42d4d959d6707f20.jpg
data/obj/Raccoon/train/raccoon-134_jpg.rf.17aaee25817c82dcccc9982cf2346af1.jpg
data/obj/Raccoon/train/raccoon-131_jpg.rf.abdd5f18ff49a86f887e2de9fb682bfa.jpg
data/obj/Raccoon/train/raccoon-28_jpg.rf.7b2be1128d5b1d3d43776e878d595a10.jpg
data/obj/Raccoon/train/raccoon-171_jpg.rf.e00ec73c6a64a07c1bc0035fa0fbca18.jpg
data/obj/Raccoon/train/raccoon-45_jpg.rf.fa6b0fabef2ed4f9e9dd6b164792d373.jpg
data/obj/Raccoon/train/raccoon-26_jpg.rf.ab207a81a546038c33a728a96bf86d14.jpg
data/obj/Raccoon/train/raccoon-137_jpg.rf.a67b07d859ac058643915ff7161e6c90.jpg
data/obj/Raccoon/train/raccoon-100_jpg.rf.e9338202a924dc489ffd94e0291a0f39.jpg
data/obj/Raccoon/train/raccoon-119_jpg.rf.72a18cd6c7a2b03e30d1088780ddad94.jpg
data/obj/Raccoon/train/raccoon-182_jpg.rf.0ee4a4d4fa828093cffc43bc3ea07a8e.jpg
data/obj/Raccoon/train/raccoon-21_jpg.rf.1a912c8d6424050610135e493cfb5181.jpg
data/obj/Raccoon/train/raccoon-181_jpg.rf.ae3fcbba8b7e24

# 4. 사전 학습된 YOLOv3 모델 파일 다운로드

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-01-06 01:54:47--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  21.0MB/s    in 10s     

2022-01-06 01:54:58 (15.4 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



# 5. 학습관련 설정 파일 수정

원 설정파일 yolov3.cfg를 yolv3_custom.cfg로 복사하고

복사한 설정파일을 수정

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_custom.cfg

max_batches = 80 * 2000
max_batches = 2000
steps1 = int(max_batches*0.8)
steps2 = int(max_batches*0.9)

# Edit classes & filters
!sed -i 's/batch=1/batch=32/' cfg/yolov3_custom.cfg
!sed -i 's/subdivisions=1/subdivisions=64/' cfg/yolov3_custom.cfg
!sed -i 's/max_batches = 500200/max_batches = $max_batches/' cfg/yolov3_custom.cfg
!sed -i 's/steps=400000,450000/steps=$steps1,$steps2/' cfg/yolov3_custom.cfg

# 6. 학습 실행

학습된 모델은  /mydrive/tmp/yolov3_custom_last.weights 로 저장된다.

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.940927), count: 1, class_loss = 0.000031, iou_loss = 0.111869, total_loss = 0.111900 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 138638, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.832237), count: 1, class_loss = 0.000001, iou_loss = 0.090077, total_loss = 0.090078 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 A

## 7. 탐지 실행

In [ ]:
!./darknet detect cfg/yolov3_custom.cfg /mydrive/tmp/yolov3_custom_last.weights data/obj/laptops_labeled/1.jpg

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
train: 27%
bird: 24%
dog: 27%
sheep: 27%
bear: 26%
zebra: 25%
handbag: 29%
tie: 30%
sports ball: 25%
baseball glove: 25%
skateboard: 31%
surfboard: 30%
orange: 25%
broccoli: 28%
hot dog: 27%
pottedplant: 30%
tvmonitor: 30%
cell phone: 28%
microwave: 26%
sink: 25%
toothbrush: 24%
bowl: 25%
chair: 44%
car: 27%
motorbike: 31%
train: 31%
truck: 43%
boat: 33%
traffic light: 37%
fire hydrant: 33%
stop sign: 30%
parking meter: 41%
bird: 42%
sheep: 28%
cow: 29%
elephant: 36%
bear: 37%
zebra: 37%
backpack: 35%
umbrella: 25%
handbag: 28%
suitcase: 33%
frisbee: 29%
snowboard: 35%
kite: 40%
skateboard: 42%
surfboard: 34%
wine glass: 41%
cup: 38%
fork: 39%
knife: 27%
bowl: 27%
banana: 29%
apple: 33%
broccoli: 35%
pizza: 44%
donut: 33%
cake: 32%
pottedplant: 26%
bed: 37%
diningtable: 26%
toilet: 42%
remote: 24%
keyboard: 27%
cell phone: 33%
microwave: 38%
toaster: 38%
book: 27%
vase: 33%
scissors: 25%
teddy bear: 41%
cow: 31%
train: 28%
bench: 24%
horse: 25%
sheep

In [ ]:
pred = plt.imread('predictions.jpg')
plt.figure(figsize=(20,20))
plt.imshow(pred)
plt.show()